In [1359]:
import pandas as pd
import pandas_profiling
import numpy as np

In [1360]:
# import random
# import torch
# import numpy as np
# random.seed(0)
# np.random.seed(0)
# torch.manual_seed(0)
# torch.cuda.manual_seed(0)
# torch.backends.cudnn.deterministic = True

In [1361]:
train = pd.read_csv('train.csv', 
#                     index_col='PassengerId',
                    dtype = {'PassengerId' : 'int64',
                             'Pclass' : 'category',
                             'Name' : 'object',
                             'Sex' : 'category',
                             'Age' : 'float64',
                             'SibSp' : 'int64',
                             'Parch' : 'int64',
                             'Ticket' : 'object',
                             'Fare' : 'float64',
                             'Cabin' : 'object',
                             'Embarked' : 'category'
                            }
                   )
# train = train[train.Age > 0]
test = pd.read_csv('test.csv',
                  dtype = {'PassengerId' : 'int64',
                             'Pclass' : 'category',
                             'Name' : 'object',
                             'Sex' : 'category',
                             'Age' : 'float64',
                             'SibSp' : 'int64',
                             'Parch' : 'int64',
                             'Ticket' : 'object',
                             'Fare' : 'float64',
                             'Cabin' : 'object',
                             'Embarked' : 'category'
                            })
all_sample = pd.concat([train,test], sort=False)
# train_X = train.drop(columns = ['Survived'])
train_X = train
test_X = test

# train_y = train['Survived']

In [1362]:
all_sample

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,NaN,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,NaN,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [1363]:
def age_group(age):
    if (age > 0) & (age <= 16):
        return 1 #Child
    elif (age > 16) & (age <= 30):
        return 2    #young
    elif (age > 30) & (age <= 60):
        return 3   #adult
    elif (age > 30):
        return 4   #old
    else:
        return 0   #unknown

In [1364]:
def data_transformation(df):
    titleset = list(set(df.Name.str.findall(r',.*?\.').astype(str).str.replace('[(\[\'\, )(\.\'\])]','').array))
    df['Title'] = df.Name.str.findall(r',.*?\.').astype(str).str.replace('[(\[\'\, )(\.\'\])]','')
    df['Title'] = df['Title'].replace({'Don' : 'Mr',
                                              'Major' : 'Mr',
                                              'Capt' : 'Mr',
                                              'Jonkheer' : 'Mr',
                                              'Rev' : 'Mr',
                                              'Col' : 'Mr',
                                              'Countess' : 'Mrs',
                                              'Mme' : 'Mrs',
                                              'Mlle' : 'Miss',
                                              'Ms' : 'Miss',}).astype('category')
    df['Cabin'] = df['Cabin'].str[0].fillna('N').map(pd.DataFrame(df['Cabin'].str[0].fillna('N'))
                                                     .groupby('Cabin').size())
    df['Embarked'] = df['Embarked'].fillna('S').map(pd.DataFrame(df['Embarked'].fillna('S'))
                                                           .groupby('Embarked').size())
    df['Title'] = df['Title'].map(pd.DataFrame(df['Title'])
                                                           .groupby('Title').size())
    df['Age'] = df['Age'].apply(age_group)
    df['Family'] = df['Parch'] + df['SibSp']
    df = pd.merge(df,
                  df[['Ticket','PassengerId']].groupby('Ticket').count().rename(columns = {'PassengerId' : 'PeopleInCabin'}),
                  on = 'Ticket').drop(columns = ['Parch', 'SibSp', 'Name', 'Ticket'])
    df['Fare'] = df['Fare']/df['PeopleInCabin']
    df['Fare'] = df['Fare'].fillna(df[df.Pclass == '3']['Fare'].mean())
    df.index = df['PassengerId']
    df = df.drop(columns = ['PassengerId'])
    df['Sex'] = df['Sex'].str.get_dummies()
    return df
    

In [1365]:
all_sample = data_transformation(all_sample)

In [1366]:
train_X = all_sample[all_sample.Survived.isna() == False]
train_X

,Survived,Pclass,Sex,Age,Fare,Cabin,Embarked,Title,Family,PeopleInCabin
PassengerId,,,,,,,,,,
1,0.0,3,0,2,7.25000,1014,916,774,1,1
2,1.0,1,1,3,35.64165,94,270,198,1,2
3,1.0,3,1,2,7.92500,1014,916,264,0,1
4,1.0,1,1,3,26.55000,94,916,198,1,2
138,0.0,1,0,3,26.55000,94,916,774,1,2
...,...,...,...,...,...,...,...,...,...,...
885,0.0,3,0,2,7.05000,1014,916,774,0,1
887,0.0,2,0,2,13.00000,1014,916,774,0,1
888,1.0,1,1,2,30.00000,65,916,264,0,1


In [1367]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(train_X.drop(columns = ['Survived']), train_X['Survived']
                                                      , test_size=0.30)

In [1368]:
X_train

,Pclass,Sex,Age,Fare,Cabin,Embarked,Title,Family,PeopleInCabin
PassengerId,,,,,,,,,
296,1,0,0,27.720800,1014,270,774,0,1
617,3,0,3,4.800000,1014,916,774,2,3
87,3,0,1,6.875000,1014,916,774,4,5
383,3,0,3,7.925000,1014,916,774,0,1
161,3,0,3,8.050000,1014,916,774,1,2
...,...,...,...,...,...,...,...,...,...
443,3,0,2,7.775000,1014,916,774,1,1
626,1,0,4,32.320800,46,916,774,0,1
185,3,1,1,7.341667,1014,916,264,2,3


In [1369]:
y_train

PassengerId
296    0.0
617    0.0
87     0.0
383    0.0
161    0.0
      ... 
443    0.0
626    0.0
185    1.0
192    0.0
281    0.0
Name: Survived, Length: 623, dtype: float64

In [1370]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

# Logistic regression classifier

In [1312]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(solver='liblinear',
                            penalty= 'l1')
logreg = logreg.fit(X_train,y_train)

In [1313]:
pred_LR = logreg.predict(X_valid)
pred_LR

array([1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 0., 1., 1., 1.,
       1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
       1., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
       0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0.,
       0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0.,
       1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1.,
       0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0.,
       0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 1.,
       0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 1., 1.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0.,
       1., 0., 1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 1., 1.

In [1314]:
print('accuracy:',accuracy_score(y_valid,pred_LR))
print('precision:',precision_score(y_valid,pred_LR))
print('recall:',recall_score(y_valid,pred_LR))
print('f:',f1_score(y_valid,pred_LR))

accuracy: 0.8022388059701493
precision: 0.7766990291262136
recall: 0.7272727272727273
f: 0.751173708920188


# K-Nearest Neighbor classifier

In [1315]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedShuffleSplit, cross_val_score

from sklearn.model_selection import GridSearchCV
## trying out multiple values for k
k_range = range(1,31)
## 
weights_options=['uniform','distance']
# 
param = {'n_neighbors':k_range, 'weights':weights_options}
## Using startifiedShufflesplit. 
cv = StratifiedShuffleSplit(n_splits=10, test_size=.30, random_state=15)
# estimator = knn, param_grid = param, n_jobs = -1 to instruct scikit learn to use all available processors. 
grid_KN = GridSearchCV(KNeighborsClassifier(), param,cv=cv,verbose = False, n_jobs=-1)
## Fitting the model. 
grid_KN = grid_KN.fit(X_train,y_train).best_estimator_

In [1316]:
pred_KN = grid_KN.predict(X_valid)
pred_KN

array([1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0.,
       1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
       0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 1.,
       1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1.,
       0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0.,
       1., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1.,
       0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0.,
       0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 1., 1.

In [1317]:
print('accuracy:',accuracy_score(y_valid,pred_KN))
print('precision:',precision_score(y_valid,pred_KN))
print('recall:',recall_score(y_valid,pred_KN))
print('f:',f1_score(y_valid,pred_KN))

accuracy: 0.7873134328358209
precision: 0.7572815533980582
recall: 0.7090909090909091
f: 0.7323943661971831


# Gaussian Naive Bayes

In [1318]:
from sklearn.naive_bayes import GaussianNB

gaussian = GaussianNB()
gaussNB = gaussian.fit(X_train, y_train)


In [1319]:
pred_gaussNB = gaussian.predict(X_valid)
pred_gaussNB

array([1., 0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1.,
       1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1.,
       1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1.,
       0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0.,
       1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0.,
       0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0.,
       0., 1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1.,
       1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1.,
       1., 0., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 1.,
       1., 1., 1., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1.,
       0., 1., 1., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1., 0., 0., 1., 1.,
       1., 0., 0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0.,
       1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1.,
       0., 1., 0., 1., 1.

In [1320]:
print('accuracy:',accuracy_score(y_valid,pred_gaussNB))
print('precision:',precision_score(y_valid,pred_gaussNB))
print('recall:',recall_score(y_valid,pred_gaussNB))
print('f:',f1_score(y_valid,pred_gaussNB))

accuracy: 0.75
precision: 0.6482758620689655
recall: 0.8545454545454545
f: 0.7372549019607844


# Support Vector Machines(SVM)

In [1321]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit, cross_val_score

Cs = [0.001, 0.01, 0.1, 1,1.5,2,2.5,3,4,5, 10] ## penalty parameter C for the error term. 
gammas = [0.0001,0.001, 0.01, 0.1, 1]
param_grid = {'C': Cs, 'gamma' : gammas}
cv = StratifiedShuffleSplit(n_splits=10, test_size=.30, random_state=15)
grid_search = GridSearchCV(SVC(kernel = 'rbf', probability=True), param_grid, cv=cv) 
grid_SVC = grid_search.fit(X_train, y_train).best_estimator_

In [1322]:
pred_SVC = grid_SVC.predict(X_valid)
pred_SVC

array([1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1.,
       1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
       1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
       0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0.,
       0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1.,
       1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1.,
       0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0.,
       1., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 1.,
       0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 1., 1.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0.,
       1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 1., 1., 1.

In [1323]:
print('accuracy:',accuracy_score(y_valid,pred_SVC))
print('precision:',precision_score(y_valid,pred_SVC))
print('recall:',recall_score(y_valid,pred_SVC))
print('f:',f1_score(y_valid,pred_SVC))

accuracy: 0.7873134328358209
precision: 0.7226890756302521
recall: 0.7818181818181819
f: 0.7510917030567685


# Decision Tree Classifier

In [1324]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold

max_depth = range(1,30)
max_feature = ['auto']
criterion=["entropy", "gini"]

param = {'max_depth':max_depth, 
         'max_features':max_feature, 
         'criterion': criterion}
grid_DT = GridSearchCV(DecisionTreeClassifier(), 
                                param_grid = param,
                                 cv=StratifiedShuffleSplit(n_splits=10, test_size=.30, random_state=15),
                                n_jobs = -1)
grid_DT = grid_DT.fit(X_train, y_train).best_estimator_

In [1325]:
pred_DT = grid_DT.predict(X_valid)
pred_DT

array([1., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 1., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1.,
       0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
       0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0.,
       0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0.,
       0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1.,
       0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0.,
       1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0.,
       0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 1.

In [1326]:
print('accuracy:',accuracy_score(y_valid,pred_DT))
print('precision:',precision_score(y_valid,pred_DT))
print('recall:',recall_score(y_valid,pred_DT))
print('f:',f1_score(y_valid,pred_DT))

accuracy: 0.7835820895522388
precision: 0.8170731707317073
recall: 0.6090909090909091
f: 0.6979166666666666


# Random forest classifier

In [1327]:
%%time
from sklearn.model_selection import GridSearchCV, StratifiedKFold, StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
n_estimators = [140,145,150,155,160];
max_depth = range(1,10);
criterions = ['gini', 'entropy'];
cv = StratifiedShuffleSplit(n_splits=10, test_size=.30, random_state=15)


parameters = {'n_estimators':n_estimators,
              'max_depth':max_depth,
              'criterion': criterions
              
        }
grid_RF = GridSearchCV(estimator=RandomForestClassifier(max_features='auto'),
                                 param_grid=parameters,
                                 cv=cv,
                                 n_jobs = -1)
grid_RF = grid_RF.fit(X_train, y_train).best_estimator_


Wall time: 25.5 s


In [1328]:
pred_RF = grid_RF.predict(X_valid)

In [1329]:
pred_RF

array([1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 1., 1.,
       1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
       1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0.,
       0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
       0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0.,
       0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1.,
       1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1.,
       0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0.,
       0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 1.,
       0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0.,
       1., 0., 1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 1., 1.

In [1330]:
print('accuracy:',accuracy_score(y_valid,pred_RF))
print('precision:',precision_score(y_valid,pred_RF))
print('recall:',recall_score(y_valid,pred_RF))
print('f:',f1_score(y_valid,pred_RF))


accuracy: 0.8097014925373134
precision: 0.7864077669902912
recall: 0.7363636363636363
f: 0.76056338028169


# Bagging Classifier

In [1331]:
from sklearn.ensemble import BaggingClassifier
n_estimators = [10,30,50,70,80,150,160, 170,175,180,185];
cv = StratifiedShuffleSplit(n_splits=10, test_size=.30, random_state=15)

parameters = {'n_estimators':n_estimators,
              
        }
grid_BC = GridSearchCV(BaggingClassifier(base_estimator= None, ## If None, then the base estimator is a decision tree.
                                      bootstrap_features=False),
                                 param_grid=parameters,
                                 cv=cv,
                                 n_jobs = -1)
grid_BC = grid_BC.fit(X_train, y_train).best_estimator_

In [1332]:
pred_BC = grid_BC.predict(X_valid)
pred_BC

array([1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0.,
       1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1.,
       0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
       0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1.,
       1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
       0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0.,
       0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1.,
       0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1.,
       1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0.,
       0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1.,
       0., 0., 0., 1., 1.

In [1333]:
print('accuracy:',accuracy_score(y_valid,pred_BC))
print('precision:',precision_score(y_valid,pred_BC))
print('recall:',recall_score(y_valid,pred_BC))
print('f:',f1_score(y_valid,pred_BC))

accuracy: 0.7873134328358209
precision: 0.7789473684210526
recall: 0.6727272727272727
f: 0.7219512195121951


# AdaBoost

In [1334]:
from sklearn.ensemble import AdaBoostClassifier
n_estimators = [100,140,145,150,160, 170,175,180,185];
cv = StratifiedShuffleSplit(n_splits=10, test_size=.30, random_state=15)
learning_r = [0.1,1,0.01,0.5]

parameters = {'n_estimators':n_estimators,
              'learning_rate':learning_r
              
        }
grid_AdaB = GridSearchCV(AdaBoostClassifier(base_estimator= None, ## If None, then the base estimator is a decision tree.
                                     ),
                                 param_grid=parameters,
                                 cv=cv,
                                 n_jobs = -1)
grid_AdaB = grid_AdaB.fit(X_train, y_train).best_estimator_

In [1335]:
pred_BC = grid_BC.predict(X_valid)
pred_BC

array([1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0.,
       1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1.,
       0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
       0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1.,
       1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
       0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0.,
       0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1.,
       0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1.,
       1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0.,
       0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1.,
       0., 0., 0., 1., 1.

In [1336]:
print('accuracy:',accuracy_score(y_valid,pred_BC))
print('precision:',precision_score(y_valid,pred_BC))
print('recall:',recall_score(y_valid,pred_BC))
print('f:',f1_score(y_valid,pred_BC))

accuracy: 0.7873134328358209
precision: 0.7789473684210526
recall: 0.6727272727272727
f: 0.7219512195121951


# Gradient Boosting Classifier

In [1337]:
from sklearn.ensemble import GradientBoostingClassifier

gradient_boost = GradientBoostingClassifier()
GBS = gradient_boost.fit(X_train, y_train)

In [1338]:
pred_GBS = GBS.predict(X_valid)
pred_GBS

array([1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1.,
       1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0.,
       0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
       0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1.,
       1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1.,
       0., 0., 1., 0., 1., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0.,
       0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 1.,
       0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1.,
       1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0.,
       0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 1., 1.

In [1339]:
print('accuracy:',accuracy_score(y_valid,pred_GBS))
print('precision:',precision_score(y_valid,pred_GBS))
print('recall:',recall_score(y_valid,pred_GBS))
print('f:',f1_score(y_valid,pred_GBS))

accuracy: 0.8208955223880597
precision: 0.81
recall: 0.7363636363636363
f: 0.7714285714285715


# XGBoost

In [1340]:
from xgboost import XGBClassifier

XGBClassifier = XGBClassifier()
XGB = XGBClassifier.fit(X_train.astype('int64'), y_train)

In [1341]:
pred_XGB = XGB.predict(X_valid.astype('int64'))
pred_XGB

array([1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 1., 1.,
       1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
       1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0.,
       0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
       0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0.,
       0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1.,
       1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 1.,
       0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 1.,
       0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0.,
       1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 1., 1.

In [1342]:
print('accuracy:',accuracy_score(y_valid,pred_XGB))
print('precision:',precision_score(y_valid,pred_XGB))
print('recall:',recall_score(y_valid,pred_XGB))
print('f:',f1_score(y_valid,pred_XGB))

accuracy: 0.8208955223880597
precision: 0.7980769230769231
recall: 0.7545454545454545
f: 0.7757009345794391


# catboost

In [1371]:
from catboost import CatBoostClassifier

cate_features_index = np.where(X_train.dtypes != float)[0]

CatBoostClassifier = CatBoostClassifier(eval_metric='Accuracy',use_best_model=True)
CatBoost = CatBoostClassifier.fit(X_train, y_train, cat_features=cate_features_index,eval_set=(X_valid,y_valid))

Learning rate set to 0.028193
0:	learn: 0.8009631	test: 0.7537313	best: 0.7537313 (0)	total: 19.7ms	remaining: 19.7s
1:	learn: 0.8025682	test: 0.7537313	best: 0.7537313 (0)	total: 22.1ms	remaining: 11s
2:	learn: 0.8025682	test: 0.7537313	best: 0.7537313 (0)	total: 35.4ms	remaining: 11.8s
3:	learn: 0.8218299	test: 0.7873134	best: 0.7873134 (3)	total: 43.8ms	remaining: 10.9s
4:	learn: 0.8234350	test: 0.7873134	best: 0.7873134 (3)	total: 65ms	remaining: 12.9s
5:	learn: 0.8202247	test: 0.7873134	best: 0.7873134 (3)	total: 67ms	remaining: 11.1s
6:	learn: 0.8234350	test: 0.7873134	best: 0.7873134 (3)	total: 75.4ms	remaining: 10.7s
7:	learn: 0.8186196	test: 0.7873134	best: 0.7873134 (3)	total: 84.8ms	remaining: 10.5s
8:	learn: 0.8186196	test: 0.7649254	best: 0.7873134 (3)	total: 101ms	remaining: 11.1s
9:	learn: 0.8186196	test: 0.7649254	best: 0.7873134 (3)	total: 104ms	remaining: 10.3s
10:	learn: 0.8202247	test: 0.7649254	best: 0.7873134 (3)	total: 104ms	remaining: 9.39s
11:	learn: 0.8154093	

94:	learn: 0.8555377	test: 0.8171642	best: 0.8171642 (27)	total: 1.09s	remaining: 10.4s
95:	learn: 0.8555377	test: 0.8171642	best: 0.8171642 (27)	total: 1.11s	remaining: 10.4s
96:	learn: 0.8571429	test: 0.8171642	best: 0.8171642 (27)	total: 1.11s	remaining: 10.4s
97:	learn: 0.8571429	test: 0.8171642	best: 0.8171642 (27)	total: 1.13s	remaining: 10.4s
98:	learn: 0.8571429	test: 0.8171642	best: 0.8171642 (27)	total: 1.14s	remaining: 10.3s
99:	learn: 0.8571429	test: 0.8171642	best: 0.8171642 (27)	total: 1.16s	remaining: 10.4s
100:	learn: 0.8571429	test: 0.8171642	best: 0.8171642 (27)	total: 1.18s	remaining: 10.5s
101:	learn: 0.8571429	test: 0.8171642	best: 0.8171642 (27)	total: 1.2s	remaining: 10.6s
102:	learn: 0.8587480	test: 0.8171642	best: 0.8171642 (27)	total: 1.21s	remaining: 10.6s
103:	learn: 0.8571429	test: 0.8171642	best: 0.8171642 (27)	total: 1.22s	remaining: 10.5s
104:	learn: 0.8587480	test: 0.8171642	best: 0.8171642 (27)	total: 1.24s	remaining: 10.6s
105:	learn: 0.8587480	test: 

187:	learn: 0.8876404	test: 0.8134328	best: 0.8208955 (183)	total: 2.46s	remaining: 10.6s
188:	learn: 0.8876404	test: 0.8134328	best: 0.8208955 (183)	total: 2.48s	remaining: 10.6s
189:	learn: 0.8892456	test: 0.8134328	best: 0.8208955 (183)	total: 2.49s	remaining: 10.6s
190:	learn: 0.8892456	test: 0.8134328	best: 0.8208955 (183)	total: 2.5s	remaining: 10.6s
191:	learn: 0.8876404	test: 0.8134328	best: 0.8208955 (183)	total: 2.5s	remaining: 10.5s
192:	learn: 0.8876404	test: 0.8134328	best: 0.8208955 (183)	total: 2.5s	remaining: 10.5s
193:	learn: 0.8876404	test: 0.8134328	best: 0.8208955 (183)	total: 2.53s	remaining: 10.5s
194:	learn: 0.8876404	test: 0.8134328	best: 0.8208955 (183)	total: 2.54s	remaining: 10.5s
195:	learn: 0.8876404	test: 0.8134328	best: 0.8208955 (183)	total: 2.54s	remaining: 10.4s
196:	learn: 0.8876404	test: 0.8134328	best: 0.8208955 (183)	total: 2.54s	remaining: 10.4s
197:	learn: 0.8876404	test: 0.8134328	best: 0.8208955 (183)	total: 2.57s	remaining: 10.4s
198:	learn: 0

279:	learn: 0.9069021	test: 0.8097015	best: 0.8208955 (183)	total: 3.77s	remaining: 9.69s
280:	learn: 0.9085072	test: 0.8134328	best: 0.8208955 (183)	total: 3.78s	remaining: 9.68s
281:	learn: 0.9085072	test: 0.8134328	best: 0.8208955 (183)	total: 3.79s	remaining: 9.65s
282:	learn: 0.9085072	test: 0.8134328	best: 0.8208955 (183)	total: 3.8s	remaining: 9.62s
283:	learn: 0.9085072	test: 0.8134328	best: 0.8208955 (183)	total: 3.81s	remaining: 9.62s
284:	learn: 0.9101124	test: 0.8134328	best: 0.8208955 (183)	total: 3.84s	remaining: 9.63s
285:	learn: 0.9101124	test: 0.8134328	best: 0.8208955 (183)	total: 3.86s	remaining: 9.63s
286:	learn: 0.9101124	test: 0.8134328	best: 0.8208955 (183)	total: 3.88s	remaining: 9.64s
287:	learn: 0.9101124	test: 0.8134328	best: 0.8208955 (183)	total: 3.9s	remaining: 9.64s
288:	learn: 0.9101124	test: 0.8134328	best: 0.8208955 (183)	total: 3.91s	remaining: 9.61s
289:	learn: 0.9085072	test: 0.8134328	best: 0.8208955 (183)	total: 3.93s	remaining: 9.62s
290:	learn: 

371:	learn: 0.9197432	test: 0.8134328	best: 0.8208955 (183)	total: 5.43s	remaining: 9.17s
372:	learn: 0.9213483	test: 0.8134328	best: 0.8208955 (183)	total: 5.45s	remaining: 9.16s
373:	learn: 0.9197432	test: 0.8134328	best: 0.8208955 (183)	total: 5.47s	remaining: 9.16s
374:	learn: 0.9213483	test: 0.8134328	best: 0.8208955 (183)	total: 5.49s	remaining: 9.15s
375:	learn: 0.9197432	test: 0.8134328	best: 0.8208955 (183)	total: 5.5s	remaining: 9.13s
376:	learn: 0.9213483	test: 0.8134328	best: 0.8208955 (183)	total: 5.53s	remaining: 9.13s
377:	learn: 0.9213483	test: 0.8134328	best: 0.8208955 (183)	total: 5.54s	remaining: 9.12s
378:	learn: 0.9213483	test: 0.8134328	best: 0.8208955 (183)	total: 5.57s	remaining: 9.12s
379:	learn: 0.9229535	test: 0.8134328	best: 0.8208955 (183)	total: 5.58s	remaining: 9.11s
380:	learn: 0.9229535	test: 0.8134328	best: 0.8208955 (183)	total: 5.6s	remaining: 9.1s
381:	learn: 0.9229535	test: 0.8134328	best: 0.8208955 (183)	total: 5.63s	remaining: 9.1s
382:	learn: 0.

...

555:	learn: 0.9502408	test: 0.8134328	best: 0.8208955 (183)	total: 8.82s	remaining: 7.04s
556:	learn: 0.9502408	test: 0.8134328	best: 0.8208955 (183)	total: 8.84s	remaining: 7.03s
557:	learn: 0.9502408	test: 0.8134328	best: 0.8208955 (183)	total: 8.86s	remaining: 7.01s
558:	learn: 0.9502408	test: 0.8134328	best: 0.8208955 (183)	total: 8.87s	remaining: 7s
559:	learn: 0.9502408	test: 0.8134328	best: 0.8208955 (183)	total: 8.9s	remaining: 6.99s
560:	learn: 0.9502408	test: 0.8134328	best: 0.8208955 (183)	total: 8.91s	remaining: 6.97s
561:	learn: 0.9502408	test: 0.8134328	best: 0.8208955 (183)	total: 8.93s	remaining: 6.96s
562:	learn: 0.9502408	test: 0.8134328	best: 0.8208955 (183)	total: 8.94s	remaining: 6.94s
563:	learn: 0.9502408	test: 0.8134328	best: 0.8208955 (183)	total: 8.95s	remaining: 6.92s
564:	learn: 0.9502408	test: 0.8134328	best: 0.8208955 (183)	total: 8.97s	remaining: 6.91s
565:	learn: 0.9502408	test: 0.8134328	best: 0.8208955 (183)	total: 8.99s	remaining: 6.89s
566:	learn: 0.

647:	learn: 0.9598716	test: 0.8059701	best: 0.8208955 (183)	total: 10.6s	remaining: 5.74s
648:	learn: 0.9630819	test: 0.8059701	best: 0.8208955 (183)	total: 10.6s	remaining: 5.72s
649:	learn: 0.9630819	test: 0.8059701	best: 0.8208955 (183)	total: 10.6s	remaining: 5.71s
650:	learn: 0.9630819	test: 0.8059701	best: 0.8208955 (183)	total: 10.6s	remaining: 5.69s
651:	learn: 0.9630819	test: 0.8097015	best: 0.8208955 (183)	total: 10.6s	remaining: 5.68s
652:	learn: 0.9662921	test: 0.8059701	best: 0.8208955 (183)	total: 10.7s	remaining: 5.66s
653:	learn: 0.9662921	test: 0.8097015	best: 0.8208955 (183)	total: 10.7s	remaining: 5.65s
654:	learn: 0.9662921	test: 0.8097015	best: 0.8208955 (183)	total: 10.7s	remaining: 5.63s
655:	learn: 0.9662921	test: 0.8059701	best: 0.8208955 (183)	total: 10.7s	remaining: 5.62s
656:	learn: 0.9662921	test: 0.8059701	best: 0.8208955 (183)	total: 10.7s	remaining: 5.61s
657:	learn: 0.9662921	test: 0.8059701	best: 0.8208955 (183)	total: 10.8s	remaining: 5.59s
658:	learn

739:	learn: 0.9743178	test: 0.8059701	best: 0.8208955 (183)	total: 12.3s	remaining: 4.32s
740:	learn: 0.9743178	test: 0.8097015	best: 0.8208955 (183)	total: 12.3s	remaining: 4.31s
741:	learn: 0.9743178	test: 0.8097015	best: 0.8208955 (183)	total: 12.4s	remaining: 4.29s
742:	learn: 0.9743178	test: 0.8097015	best: 0.8208955 (183)	total: 12.4s	remaining: 4.28s
743:	learn: 0.9743178	test: 0.8097015	best: 0.8208955 (183)	total: 12.4s	remaining: 4.26s
744:	learn: 0.9743178	test: 0.8097015	best: 0.8208955 (183)	total: 12.4s	remaining: 4.25s
745:	learn: 0.9743178	test: 0.8097015	best: 0.8208955 (183)	total: 12.4s	remaining: 4.23s
746:	learn: 0.9743178	test: 0.8097015	best: 0.8208955 (183)	total: 12.4s	remaining: 4.21s
747:	learn: 0.9743178	test: 0.8097015	best: 0.8208955 (183)	total: 12.5s	remaining: 4.2s
748:	learn: 0.9743178	test: 0.8097015	best: 0.8208955 (183)	total: 12.5s	remaining: 4.18s
749:	learn: 0.9743178	test: 0.8097015	best: 0.8208955 (183)	total: 12.5s	remaining: 4.17s
750:	learn:

831:	learn: 0.9759230	test: 0.8097015	best: 0.8208955 (183)	total: 14.1s	remaining: 2.84s
832:	learn: 0.9759230	test: 0.8097015	best: 0.8208955 (183)	total: 14.1s	remaining: 2.83s
833:	learn: 0.9759230	test: 0.8097015	best: 0.8208955 (183)	total: 14.1s	remaining: 2.81s
834:	learn: 0.9759230	test: 0.8097015	best: 0.8208955 (183)	total: 14.1s	remaining: 2.79s
835:	learn: 0.9759230	test: 0.8097015	best: 0.8208955 (183)	total: 14.1s	remaining: 2.77s
836:	learn: 0.9759230	test: 0.8097015	best: 0.8208955 (183)	total: 14.2s	remaining: 2.76s
837:	learn: 0.9759230	test: 0.8097015	best: 0.8208955 (183)	total: 14.2s	remaining: 2.74s
838:	learn: 0.9759230	test: 0.8097015	best: 0.8208955 (183)	total: 14.2s	remaining: 2.73s
839:	learn: 0.9759230	test: 0.8097015	best: 0.8208955 (183)	total: 14.2s	remaining: 2.71s
840:	learn: 0.9759230	test: 0.8097015	best: 0.8208955 (183)	total: 14.3s	remaining: 2.69s
841:	learn: 0.9759230	test: 0.8097015	best: 0.8208955 (183)	total: 14.3s	remaining: 2.68s
842:	learn

923:	learn: 0.9775281	test: 0.8097015	best: 0.8208955 (183)	total: 15.9s	remaining: 1.31s
924:	learn: 0.9775281	test: 0.8097015	best: 0.8208955 (183)	total: 16s	remaining: 1.29s
925:	learn: 0.9775281	test: 0.8097015	best: 0.8208955 (183)	total: 16s	remaining: 1.28s
926:	learn: 0.9775281	test: 0.8097015	best: 0.8208955 (183)	total: 16s	remaining: 1.26s
927:	learn: 0.9775281	test: 0.8097015	best: 0.8208955 (183)	total: 16s	remaining: 1.24s
928:	learn: 0.9775281	test: 0.8097015	best: 0.8208955 (183)	total: 16s	remaining: 1.22s
929:	learn: 0.9775281	test: 0.8097015	best: 0.8208955 (183)	total: 16s	remaining: 1.21s
930:	learn: 0.9775281	test: 0.8097015	best: 0.8208955 (183)	total: 16.1s	remaining: 1.19s
931:	learn: 0.9775281	test: 0.8097015	best: 0.8208955 (183)	total: 16.1s	remaining: 1.17s
932:	learn: 0.9775281	test: 0.8097015	best: 0.8208955 (183)	total: 16.1s	remaining: 1.16s
933:	learn: 0.9775281	test: 0.8097015	best: 0.8208955 (183)	total: 16.1s	remaining: 1.14s
934:	learn: 0.9775281	

In [1372]:
pred_CatBoost = CatBoost.predict(X_valid)
pred_CatBoost

array([0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0.,
       1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1.,
       0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0.,
       0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0.,
       0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0.,
       1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0.,
       0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0.,
       1., 1., 0., 0., 1.

In [1373]:
print('accuracy:',accuracy_score(y_valid,pred_CatBoost))
print('precision:',precision_score(y_valid,pred_CatBoost))
print('recall:',recall_score(y_valid,pred_CatBoost))
print('f:',f1_score(y_valid,pred_CatBoost))

accuracy: 0.8208955223880597
precision: 0.8941176470588236
recall: 0.6608695652173913
f: 0.7599999999999999


# ExtraTreesClassifier

In [1346]:
from sklearn.ensemble import ExtraTreesClassifier

ExtraTreesClassifier = ExtraTreesClassifier(n_estimators = 100)
ETC = ExtraTreesClassifier.fit(X_train, y_train)

In [1347]:
pred_ETC = ETC.predict(X_valid)
pred_ETC

array([1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 1., 0.,
       1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1.,
       0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0.,
       0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1.,
       1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0.,
       0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1.,
       1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0.,
       0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1.,
       0., 0., 0., 1., 1.

In [1348]:
print('accuracy:',accuracy_score(y_valid,pred_ETC))
print('precision:',precision_score(y_valid,pred_ETC))
print('recall:',recall_score(y_valid,pred_ETC))
print('f:',f1_score(y_valid,pred_ETC))

accuracy: 0.7723880597014925
precision: 0.7578947368421053
recall: 0.6545454545454545
f: 0.702439024390244


# GaussianProcessClassifier

In [1349]:
from sklearn.gaussian_process import GaussianProcessClassifier
GaussianProcessClassifier = GaussianProcessClassifier()
GPC = GaussianProcessClassifier.fit(X_train, y_train)

In [1350]:
pred_GPC = GPC.predict(X_valid)
pred_GPC

array([1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 1., 1.,
       1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
       0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1.,
       0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0.,
       0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1.,
       0., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1.,
       1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0.,
       0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1.,
       0., 0., 0., 1., 1.

In [1351]:
print('accuracy:',accuracy_score(y_valid,pred_GPC))
print('precision:',precision_score(y_valid,pred_GPC))
print('recall:',recall_score(y_valid,pred_GPC))
print('f:',f1_score(y_valid,pred_GPC))

accuracy: 0.8059701492537313
precision: 0.7843137254901961
recall: 0.7272727272727273
f: 0.7547169811320754


# Voting classification

In [1352]:
from sklearn.ensemble import VotingClassifier

voting_classifier = VotingClassifier(estimators=[
    ('logidic regression', logreg),
    ('k-nearest neighbors', grid_KN),
    ('gauss naive bias', gaussNB),
    ('SVM', grid_SVC),
    ('Decision Tree', grid_DT),
    ('random_forest', grid_RF),
    ('bagging_classifier', grid_BC),
    ('AdaBoost', grid_AdaB),
    ('GradBoost', GBS),
    ('ETC', ETC),
    ('GPC', GPC)
],voting='hard')

voting_classifier = voting_classifier.fit(X_train, y_train)

In [1353]:
y_pred = voting_classifier.predict(X_valid)
voting_accy = round(accuracy_score(y_pred, y_valid), 3)
print(voting_accy)

0.81


# The Best model choose

In [1354]:
all_models = [logreg,
              grid_KN, 
              gaussNB,
              grid_SVC,
              grid_DT,
              grid_RF,
              grid_BC,
              grid_AdaB,
              GBS,
             ETC,
             GPC]

c = {}
for i in all_models:
    a = i.predict(X_valid)
    b = accuracy_score(a, y_valid)
    c[i] = b
a = XGB.predict(X_valid.astype('int64'))
c[XGB] = accuracy_score(y_valid,a)
a = CatBoost.predict(X_valid)
c['CatBoost'] = accuracy_score(y_valid,a)

max(c, key=c.get)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

# Testing

In [1374]:
X_test = all_sample[all_sample.Survived.isna() == True].drop(columns = ['Survived'])
X_test

,Pclass,Sex,Age,Fare,Cabin,Embarked,Title,Family,PeopleInCabin
PassengerId,,,,,,,,,
1126,1,0,3,35.641650,94,270,774,1,2
1038,1,0,0,25.931250,41,916,774,0,2
1281,3,0,1,4.215000,1014,916,61,4,5
1009,3,1,1,5.566667,5,916,264,2,3
947,3,0,1,4.854167,1014,123,61,5,6
...,...,...,...,...,...,...,...,...,...
1302,3,1,0,7.750000,1014,123,264,0,1
1304,3,1,2,7.775000,1014,916,264,0,1
1305,3,0,0,8.050000,1014,916,774,0,1


In [1375]:
X_test[X_test.Fare.isna()]

,Pclass,Sex,Age,Fare,Cabin,Embarked,Title,Family,PeopleInCabin
PassengerId,,,,,,,,,


In [1376]:
pred_test = CatBoost.predict(X_test)
pred_test

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0., 0.,
       0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.,
       1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0.,
       1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
       0., 1., 1., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 0., 1., 0., 0.,
       1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 1.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0.,
       0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 1.,
       1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0.,
       0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0.,
       0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [1378]:
passengerid = X_test.index
submission = pd.DataFrame({
        "PassengerId": passengerid,
        "Survived": pred_test
    })

submission.PassengerId = submission.PassengerId.astype(int)
submission.Survived = submission.Survived.astype(int)

submission.to_csv("titanic1_submission.csv", index=False)